In [45]:
from pyspark.sql import SparkSession
import pyspark
spark = SparkSession.builder.master("local[1]").appName('Tma_Tme_Digital').getOrCreate()
df = spark.read.csv("01- Código.csv", header=True, inferSchema=True)
df.createOrReplaceTempView("View")

In [51]:
# Consulta SQL usando o DataFrame df
resultado = spark.sql("""
    SELECT
        Data,
        Campanha,
        Segmento,
        SUM(Tempo_logado) AS Tempo_logado,
        SUM(Tempo_falado) AS Tempo_falado,
        SUM(Chamadas_atendidas) AS Chamadas_atendidas,
        SUM(TMA) AS TMA,
        SUM(IDLE) AS IDLE,
        COUNT(*) AS QTD
    FROM View
    WHERE Agente != 'Homologação' AND Data BETWEEN '2023-07-01' AND DATEADD(DAY, -1, CURRENT_DATE())
    GROUP BY Data, Campanha, Segmento
    HAVING SUM(IDLE) > 0
    ORDER BY 1 DESC
""")

# Exibir o resultado
resultado.show()


+----------+--------------------+--------+------------+------------+------------------+-----+-------+----+
|      Data|            Campanha|Segmento|Tempo_logado|Tempo_falado|Chamadas_atendidas|  TMA|   IDLE| QTD|
+----------+--------------------+--------+------------+------------+------------------+-----+-------+----+
|2023-09-01|AV - TIM (FLUXO O...|POS PAGO|      500884|      262544|             13701| 2660| 238340| 264|
|2023-09-01|AV - TIM CONTROLE...|Controle|     1138290|      375008|             22133| 5365| 763282| 600|
|2023-09-01|AV - TIM CONTROLE...|Controle|     3529932|     1765960|             94683|18139|1763972|1848|
|2023-09-01|AV - TIM POS PAGO...|POS PAGO|     1226577|      576645|             30685| 6381| 649932| 648|
|2023-08-31|AV - TIM CONTROLE...|Controle|     3512639|     1982348|            100917|19180|1530291|1848|
|2023-08-31|AV - TIM (FLUXO O...|POS PAGO|      493884|      322004|             15512| 2895| 171880| 264|
|2023-08-31|AV - TIM POS PAGO...|POS 

In [75]:
from pyspark.sql.functions import col, lit, sum, count, when
from datetime import datetime, timedelta
data_limite = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

# Filtrar e transformar seus dados
df_filtered = df.filter(
    (col("Data") >= "2023-07-01") &
    (col("Data") <= data_limite) &
    (col("Agente") != "Homologação")
)

# Realizar agregações
df_aggregated = df_filtered.groupBy("Data", "Campanha", "Segmento").agg(
    sum("Tempo_logado").alias("Tempo_logado"),
    sum("Tempo_falado").alias("Tempo_falado"),
    sum("Chamadas_atendidas").alias("Chamadas_atendidas"),
    sum("TMA").alias("TMA"),
    sum("IDLE").alias("IDLE"),
    count("*").alias("QTD")
)

# Filtrar apenas os grupos onde SUM(IDLE) > 0
df_filtered_groups = df_aggregated.filter(col("IDLE") > 0)

# Adicionar as colunas calculadas
df_result = df_filtered_groups.withColumn("HORA_PROD_PA_META", lit(HORA_PROD_PA_META))\
    .withColumn("VALOR_PA", lit(VALOR_PA))\
    .withColumn("TEMP_PRODUTIVO_META", col("QTD") * col("HORA_PROD_PA_META"))\
    .withColumn("PA_PRODUTIVO_REALIZADO", col("Tempo_falado") / col("HORA_PROD_PA_META"))\
    .orderBy(col("Data").desc())

# Exibir o resultado
df_result.show()

+----------+--------------------+--------+------------+------------+------------------+-----+-------+----+-----------------+--------+-------------------+----------------------+
|      Data|            Campanha|Segmento|Tempo_logado|Tempo_falado|Chamadas_atendidas|  TMA|   IDLE| QTD|HORA_PROD_PA_META|VALOR_PA|TEMP_PRODUTIVO_META|PA_PRODUTIVO_REALIZADO|
+----------+--------------------+--------+------------+------------+------------------+-----+-------+----+-----------------+--------+-------------------+----------------------+
|2023-09-01|AV - TIM (FLUXO O...|POS PAGO|      500884|      262544|             13701| 2660| 238340| 264|            40200|    8265|           10612800|     6.530945273631841|
|2023-09-01|AV - TIM CONTROLE...|Controle|     1138290|      375008|             22133| 5365| 763282| 600|            40200|    8265|           24120000|     9.328557213930349|
|2023-09-01|AV - TIM CONTROLE...|Controle|     3529932|     1765960|             94683|18139|1763972|1848|         

In [76]:

from pyspark.sql.functions import col, sum, when, lit
from pyspark.sql.window import Window


# Defina suas variáveis de contexto
HORA_PROD_PA_META = 40200
VALOR_PA = 8265

# Filtrar e transformar seus dados

# Filtrar e transformar seus dados
df_filtered = df.filter((col("Data") >= "2023-07-01") & (col("Data") <= data_limite) & (col("Agente") != "Homologação"))

# Realizar agregações
df_aggregated = df_filtered.groupBy("Data").agg(
    sum("Tempo_logado").alias("TEMP_LOGADO_TOTAL"),
    sum("Tempo_falado").alias("TEMP_PRODUTIVO_REALIZADO"),
    when((col("Data") >= "2023-07-01") & (col("Data") <= "2023-07-28"), 36).otherwise(105).alias("PA_META")
)

# Adicionar as colunas calculadas
df_result = df_aggregated.withColumn("HORA_PROD_PA_META", lit(HORA_PROD_PA_META))\
    .withColumn("VALOR_PA", lit(VALOR_PA))\
    .withColumn("TEMP_PRODUTIVO_META", col("PA_META") * col("HORA_PROD_PA_META"))\
    .withColumn("PA_PRODUTIVO_REALIZADO", col("TEMP_PRODUTIVO_REALIZADO") / col("HORA_PROD_PA_META"))\
    .orderBy("Data")

# Exibir o resultado
df_result.show()


+----------+-----------------+------------------------+-------+-----------------+--------+-------------------+----------------------+
|      Data|TEMP_LOGADO_TOTAL|TEMP_PRODUTIVO_REALIZADO|PA_META|HORA_PROD_PA_META|VALOR_PA|TEMP_PRODUTIVO_META|PA_PRODUTIVO_REALIZADO|
+----------+-----------------+------------------------+-------+-----------------+--------+-------------------+----------------------+
|2023-07-21|          4550393|                 2960851|     36|            40200|    8265|            1447200|     73.65300995024876|
|2023-07-22|          4579261|                 1992298|     36|            40200|    8265|            1447200|    49.559651741293536|
|2023-07-24|          4481923|                 1628783|     36|            40200|    8265|            1447200|     40.51699004975124|
|2023-07-25|          4524153|                 1647404|     36|            40200|    8265|            1447200|     40.98019900497513|
|2023-07-26|          4874376|                 3731530|     36

In [79]:
df_result.toPandas().value_counts()

Data        TEMP_LOGADO_TOTAL  TEMP_PRODUTIVO_REALIZADO  PA_META  HORA_PROD_PA_META  VALOR_PA  TEMP_PRODUTIVO_META  PA_PRODUTIVO_REALIZADO
2023-07-21  4550393            2960851                   36       40200              8265      1447200              73.653010                 1
2023-08-12  6384970            1425884                   105      40200              8265      4221000              35.469751                 1
2023-08-15  5502977            2374065                   105      40200              8265      4221000              59.056343                 1
2023-08-16  6305394            4648132                   105      40200              8265      4221000              115.625174                1
2023-08-17  6504005            3078093                   105      40200              8265      4221000              76.569478                 1
2023-08-18  6446438            4065393                   105      40200              8265      4221000              101.129179               